In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import MatplotlibKeyMap as mk
import os
import SanskritCharts as hc


In [2]:
df_words = pd.read_csv('./ZipfsGraph/sanskrit_words.csv')
print(df_words.head())

  word  frequency  probability
0    च       2697     0.070560
1    न       1512     0.039557
2    स       1092     0.028569
3   तु        949     0.024828
4   हि        898     0.023494


In [3]:
def calculate_ngram_frequency(n):
    df_ngram = pd.DataFrame(columns=['ngram', 'Frequency'])
    for i in range(0, len(df_words)):
        word = df_words.loc[i]['word']
        
        for j in range(0, len(word) - n + 1):
            ngram = word[j:j+n]
            
            if ngram in df_ngram['ngram'].values:
                df_ngram.loc[df_ngram['ngram'] == ngram, 'Frequency'] += df_words.loc[i]['frequency']
            else:
                df_ngram.loc[len(df_ngram)] = [ngram, df_words.loc[i]['frequency']]

    df_ngram['Probability'] = df_ngram['Frequency'] / df_ngram['Frequency'].sum()
    df_ngram.to_csv('./ngram/ngram_' + str(n) + '.csv', index=False)
    print(str(n) + "-gram frequency calculated")

In [ ]:
for i in range(1, 10):
    calculate_ngram_frequency(i)

In [12]:
ngram_dict = {}

for i in range(1, 10):
    ngram_dict[i] = pd.read_csv('./ngram/ngram_' + str(i) + '.csv')

In [13]:
total_chars = ngram_dict[1].shape[0]
print("Total chars: " + str(total_chars)) 

F0=np.log2(total_chars)

print("F0: " + str(F0))

Total chars: 54
F0: 5.754887502163468


In [14]:
Fn = []
Fn.append(F0)

In [15]:
F1 = ngram_dict[1]['Probability'].apply(lambda x: x * np.log2(x)).sum() * -1
print("F1: " + str(F1))

Fn.append(F1)

F1: 4.758469530697246


In [16]:
for i in range(2, 10):
    Fi = ngram_dict[i]['Probability'].apply(lambda x: x * np.log2(x)).sum() * -1
    Fi = Fi - Fn[len(Fn) - 1]
    print("F" + str(i) + ": " + str(Fi))
    Fn.append(Fi)

F2: 3.585025229219596
F3: 7.156750464523789
F4: 4.861391668946407
F5: 7.619893131937943
F6: 4.799881581732771
F7: 7.348874174862303
F8: 4.385085218228676
F9: 6.707589796910391


In [17]:
df = pd.DataFrame(columns=['n', 'Fn'])

for i in range(0, len(Fn)):
    df.loc[i] = [i, Fn[i]]
    
df.to_csv('./ngram/ngram_entropy.csv', index=False)